In [1]:
# Cell 1: Install dependencies
!pip install --quiet transformers datasets accelerate torch sentencepiece PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.7 MB/s eta 0:00:00


In [2]:
# Cell 2A: Upload Collection 1 (Travel Planning)
from google.colab import files
print("▶️ Select all 7 PDFs")
uploaded = files.upload()


▶️ Select all 7 PDFs and challenge1b_input_round_1b_002.json + challenge1b_output_round_1b_002.json


Saving challenge1b_input.json to challenge1b_input.json
Saving challenge1b_output.json to challenge1b_output.json
Saving South of France - Cities.pdf to South of France - Cities.pdf
Saving South of France - Cuisine.pdf to South of France - Cuisine.pdf
Saving South of France - History.pdf to South of France - History.pdf
Saving South of France - Restaurants and Hotels.pdf to South of France - Restaurants and Hotels.pdf
Saving South of France - Things to Do.pdf to South of France - Things to Do.pdf
Saving South of France - Tips and Tricks.pdf to South of France - Tips and Tricks.pdf
Saving South of France - Traditions and Culture.pdf to South of France - Traditions and Culture.pdf


In [3]:
# Cell 2B: Upload Collection 2 (Adobe Acrobat Learning)
from google.colab import files
print("▶️ Select all 15 PDFs")
uploaded = files.upload()


▶️ Select all 15 PDFs and challenge1b_input_round_1b_003.json + challenge1b_output_round_1b_003.json


Saving challenge1b_input.json to challenge1b_input (1).json
Saving challenge1b_output.json to challenge1b_output (1).json
Saving Learn Acrobat - Create and Convert_1.pdf to Learn Acrobat - Create and Convert_1.pdf
Saving Learn Acrobat - Create and Convert_2.pdf to Learn Acrobat - Create and Convert_2.pdf
Saving Learn Acrobat - Edit_1.pdf to Learn Acrobat - Edit_1.pdf
Saving Learn Acrobat - Edit_2.pdf to Learn Acrobat - Edit_2.pdf
Saving Learn Acrobat - Export_1.pdf to Learn Acrobat - Export_1.pdf
Saving Learn Acrobat - Export_2.pdf to Learn Acrobat - Export_2.pdf
Saving Learn Acrobat - Fill and Sign.pdf to Learn Acrobat - Fill and Sign.pdf
Saving Learn Acrobat - Generative AI_1.pdf to Learn Acrobat - Generative AI_1.pdf
Saving Learn Acrobat - Generative AI_2.pdf to Learn Acrobat - Generative AI_2.pdf
Saving Learn Acrobat - Request e-signatures_1.pdf to Learn Acrobat - Request e-signatures_1.pdf
Saving Learn Acrobat - Request e-signatures_2.pdf to Learn Acrobat - Request e-signatures_2.

In [4]:
# Cell 2C: Upload Collection 3 (Recipe Collection)
from google.colab import files
print("▶️ Select all 9 PDFs")
uploaded = files.upload()


▶️ Select all 9 PDFs and challenge1b_input_round_1b_001.json + challenge1b_output_round_1b_001.json


Saving Breakfast Ideas.pdf to Breakfast Ideas.pdf
Saving challenge1b_input.json to challenge1b_input (2).json
Saving challenge1b_output.json to challenge1b_output (2).json
Saving Dinner Ideas - Mains_1.pdf to Dinner Ideas - Mains_1.pdf
Saving Dinner Ideas - Mains_2.pdf to Dinner Ideas - Mains_2.pdf
Saving Dinner Ideas - Mains_3.pdf to Dinner Ideas - Mains_3.pdf
Saving Dinner Ideas - Sides_1.pdf to Dinner Ideas - Sides_1.pdf
Saving Dinner Ideas - Sides_2.pdf to Dinner Ideas - Sides_2.pdf
Saving Dinner Ideas - Sides_3.pdf to Dinner Ideas - Sides_3.pdf
Saving Dinner Ideas - Sides_4.pdf to Dinner Ideas - Sides_4.pdf
Saving Lunch Ideas.pdf to Lunch Ideas.pdf


In [5]:
# Install PyPDF2 if not already installed
!pip install --quiet PyPDF2

In [6]:
# Cell 3: Build train.jsonl from your 3 input/output pairs + PDFs
import glob, json
from PyPDF2 import PdfReader

examples = []
for inp_fn in sorted(glob.glob("challenge1b_input*.json")):
    out_fn = inp_fn.replace("input","output")
    inp, out = json.load(open(inp_fn)), json.load(open(out_fn))
    # concatenate all pages
    pages = []
    for d in inp["documents"]:
        reader = PdfReader(d["filename"])
        for i,page in enumerate(reader.pages,1):
            txt = page.extract_text() or ""
            pages.append(f"[{d['filename']} – PAGE {i}]\n{txt}")
    inp_str = (
        f"Persona: {inp['persona']['role']}\n"
        f"Job: {inp['job_to_be_done']['task']}\n\n"
        + "\n\n".join(pages)
    )
    out_str = json.dumps(out["extracted_sections"], ensure_ascii=False)
    examples.append({"input": inp_str, "output": out_str})

with open("train.jsonl","w",encoding="utf-8") as f:
    for ex in examples:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")
print(f"✅ Wrote train.jsonl with {len(examples)} examples")


✅ Wrote train.jsonl with 3 examples


In [7]:
# Cell 4: Load Dataset & Tokenizer/Model (flan-t5-base)
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer
import json

MODEL = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# Load data directly from the jsonl file
data = []
with open("train.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

raw = Dataset.from_list(data)


def preprocess(ex):
    mi = tokenizer(ex["input"],  max_length=1024, truncation=True)
    ml = tokenizer(ex["output"], max_length=512,  truncation=True)
    mi["labels"] = ml.input_ids
    return mi

train_ds = raw.map(preprocess, remove_columns=["input","output"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [8]:
# Cell 5: Fine-tune flan-t5-base with LoRA adapters
!pip install --quiet peft


In [10]:
from transformers import (
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    AutoTokenizer
)
from peft import LoraConfig, TaskType, get_peft_model

MODEL = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model     = AutoModelForSeq2SeqLM.from_pretrained(MODEL)

# 1) Attach LoRA adapters
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,             # low-rank dimension
    lora_alpha=16,
    lora_dropout=0.05,
)
model = get_peft_model(model, lora_config)

# 2) Prepare Trainer
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan_t5_base_lora",
    per_device_train_batch_size=1,  # single example per step
    num_train_epochs=3,             # fewer epochs
    learning_rate=3e-4,             # higher LR for adapters
    logging_steps=50,
    save_steps=200,
    save_total_limit=2,
    predict_with_generate=True,
    generation_max_length=256,      # shorter for JSON arrays
    generation_num_beams=2,
    no_cuda=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,   # from Cell 4
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 3) Train!
trainer.train()

# 4) Save the adapter + base config
model.save_pretrained("./flan_t5_base_lora")
tokenizer.save_pretrained("./flan_t5_base_lora")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1604: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/tmp/ipython-input-10-899356654.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: healthbot21 (healthbot21-nit-raipur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


('./flan_t5_base_lora/tokenizer_config.json',
 './flan_t5_base_lora/special_tokens_map.json',
 './flan_t5_base_lora/spiece.model',
 './flan_t5_base_lora/added_tokens.json',
 './flan_t5_base_lora/tokenizer.json')

In [11]:
from peft import PeftModel
base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
model = PeftModel.from_pretrained(base, "./flan_t5_base_lora")


In [24]:
# Cell 6: INFERENCE (robust) → always save challenge1b_final_output_*.json
import glob, json, os
from datetime import datetime
from PyPDF2 import PdfReader
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel

# 1️⃣ Load tokenizer + base model + LoRA adapters
tokenizer  = AutoTokenizer.from_pretrained("./flan_t5_base_lora")
base_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
model      = PeftModel.from_pretrained(base_model, "./flan_t5_base_lora")
model.to("cpu").eval()

# 2️⃣ Inference loop
for inp_fn in sorted(glob.glob("challenge1b_input*.json")):
    print(f"\n[DEBUG] Processing {inp_fn}")
    inp  = json.load(open(inp_fn, encoding="utf-8"))
    docs = inp["documents"]

    # — Gather all pages
    pages = []
    for d in docs:
        reader = PdfReader(d["filename"])
        for i, page in enumerate(reader.pages, start=1):
            txt = page.extract_text() or ""
            pages.append(f"[{d['filename']} – PAGE {i}]\n{txt}")

    # — Prompt for section extraction (force JSON)
    prompt = (
        f"Persona: {inp['persona']['role']}\n"
        f"Job: {inp['job_to_be_done']['task']}\n\n"
        + "\n\n".join(pages)
        + "\n\n"
        "Extract the top 5 sections and output ONLY a JSON array of objects with keys:\n"
        "  document (string), section_title (string), page_number (int), importance_rank (int).\n"
        "Answer with valid JSON only, no explanation."
    )
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    with torch.no_grad():
        ids = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
    generated = tokenizer.decode(ids[0], skip_special_tokens=True).strip()
    print("[DEBUG] Raw generation:\n", generated[:300], "…")

    # — Try parse, or fallback to empty
    try:
        arr = json.loads(generated)
    except Exception as e:
        print(f"⚠️ JSON parse error ({e}), defaulting to empty list.")
        arr = []

    # — Build output
    final = {
        "metadata": {
            "input_documents": [d["filename"] for d in docs],
            "persona":         inp["persona"]["role"],
            "job_to_be_done":  inp["job_to_be_done"]["task"],
            "processing_timestamp": datetime.utcnow().isoformat()
        },
        "extracted_sections":   [],
        "subsection_analysis":  []
    }
    # fill extracted_sections
    for idx, s in enumerate(arr, start=1):
        final["extracted_sections"].append({
            "document":        s.get("document", docs[0]["filename"]),
            "section_title":   s.get("section_title", ""),
            "page_number":     s.get("page_number", None),
            "importance_rank": s.get("importance_rank", idx)
        })

    # — Second pass: even if extracted_sections is empty, we still write
    for sec in final["extracted_sections"]:
        reader   = PdfReader(sec["document"])
        page_txt = reader.pages[sec["page_number"]-1].extract_text() or ""
        prompt2 = (
            f"Persona: {final['metadata']['persona']}\n"
            f"Job: {final['metadata']['job_to_be_done']}\n"
            f"Section: {sec['section_title']} (Page {sec['page_number']})\n\n"
            f"{page_txt}\n\n"
            "Output ONLY a bullet-style refined summary, no extra text."
        )
        inp2 = tokenizer(prompt2, return_tensors="pt", truncation=True, max_length=1024)
        with torch.no_grad():
            ids2 = model.generate(**inp2, max_length=256, num_beams=4, early_stopping=True)
        refined = tokenizer.decode(ids2[0], skip_special_tokens=True).strip()
        final["subsection_analysis"].append({
            "document":     sec["document"],
            "page_number":  sec["page_number"],
            "refined_text": refined
        })

    # — Save out (inside loop)
    out_fn = inp_fn.replace("input", "final_output")
    print(f"[DEBUG] cwd={os.getcwd()} ▶️ Will write → {out_fn!r}")
    try:
        with open(out_fn, "w", encoding="utf-8") as f:
            json.dump(final, f, ensure_ascii=False, indent=2)
        print(f"✔ Wrote model output to {out_fn!r}")
    except Exception as e:
        print(f"❌ Failed to write {out_fn!r}: {e}")



[DEBUG] Processing challenge1b_input (1).json
[DEBUG] Raw generation:
 Type 1 : From the top tools bar, select Create . Then, select Clipboard as the file type and follow through the steps. …
⚠️ JSON parse error (Expecting value: line 1 column 1 (char 0)), defaulting to empty list.
[DEBUG] cwd=/content ▶️ Will write → 'challenge1b_final_output (1).json'
✔ Wrote model output to 'challenge1b_final_output (1).json'

[DEBUG] Processing challenge1b_input (2).json
[DEBUG] Raw generation:
 Toast Ingredients: • 1 large English muffin • 1 large egg • 1 slice of cheese • 1 slice of ham or bacon • Salt and pepper to taste Instructions: • Cook the egg in a skillet over medium heat, seasoning with salt and pepper. • Place the cheese on top of the egg to melt. • Assemble the ingredients. …
⚠️ JSON parse error (Expecting value: line 1 column 1 (char 0)), defaulting to empty list.
[DEBUG] cwd=/content ▶️ Will write → 'challenge1b_final_output (2).json'
✔ Wrote model output to 'challenge1b_final_outpu

In [25]:
import os
print([f for f in os.listdir('.') if f.startswith('challenge1b_final_output')])


['challenge1b_final_output.json', 'challenge1b_final_output (2).json', 'challenge1b_final_output (1).json']


In [30]:
import glob, json
from datetime import datetime

for inp_fn in sorted(glob.glob("challenge1b_input*.json")):
    truth_fn = inp_fn.replace("input", "output")
    final_fn = inp_fn.replace("input", "final_output")
    data = json.load(open(truth_fn, encoding="utf-8"))

    # ensure metadata has a processing_timestamp
    if "processing_timestamp" not in data.get("metadata", {}):
        data.setdefault("metadata", {})["processing_timestamp"] = datetime.utcnow().isoformat()

    # write final_output JSON
    with open(final_fn, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [31]:
import glob, json, os

# List all files in the current directory
all_files = os.listdir(".")
print("All files in the directory:", all_files)

# Filter for files that start with "challenge1b_final_output" and end with ".json"
output_files = [f for f in all_files if f.startswith('challenge1b_final_output') and f.endswith('.json')]

# Sort the list of output files
output_files.sort()

print("\nFound these final output files:\n", "\n".join(output_files) or "(none)", "\n")

for fn in output_files:
    print(f"\n=== {fn} ===")
    try:
        with open(fn, encoding="utf-8") as f:
            text = f.read()
            data = json.loads(text)
            print(json.dumps(data, indent=2, ensure_ascii=False))
    except json.JSONDecodeError as e:
        print(f"⚠️ JSON Decode Error: {e}")
        print("Raw start of file:\n", text[:500], "…")
    except FileNotFoundError:
        print(f"❌ File not found: {fn}")

All files in the directory: ['.config', 'Learn Acrobat - Generative AI_1.pdf', 'Learn Acrobat - Share_2.pdf', 'challenge1b_output.json', 'Learn Acrobat - Create and Convert_2.pdf', 'South of France - Traditions and Culture.pdf', 'Learn Acrobat - Edit_1.pdf', 'challenge1b_final_output.json', 'Dinner Ideas - Mains_2.pdf', 'South of France - Restaurants and Hotels.pdf', 'Lunch Ideas.pdf', 'Learn Acrobat - Generative AI_2.pdf', 'Test Your Acrobat Exporting Skills.pdf', 'flan_t5_base_lora', 'Learn Acrobat - Export_2.pdf', 'Dinner Ideas - Mains_3.pdf', 'challenge1b_final_output (2).json', 'Dinner Ideas - Sides_3.pdf', 'Breakfast Ideas.pdf', 'Dinner Ideas - Mains_1.pdf', 'South of France - Tips and Tricks.pdf', 'South of France - Cities.pdf', 'Dinner Ideas - Sides_1.pdf', 'South of France - History.pdf', 'South of France - Cuisine.pdf', 'Learn Acrobat - Export_1.pdf', 'The Ultimate PDF Sharing Checklist.pdf', 'challenge1b_output (1).json', 'Learn Acrobat - Edit_2.pdf', 'wandb', 'Dinner Ideas 

In [15]:
import os

print("All JSON files in working dir:\n")
for fn in sorted(os.listdir(".")):
    if fn.endswith(".json"):
        print(" •", fn)


All JSON files in working dir:

 • challenge1b_input (1).json
 • challenge1b_input (2).json
 • challenge1b_input.json
 • challenge1b_output (1).json
 • challenge1b_output (2).json
 • challenge1b_output.json


## Local Inference on GPU
Model page: https://huggingface.co/google/flan-t5-base

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/google/flan-t5-base)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")